In [1]:
#using Pkg
#Pkg.activate("D:\\Science\\Code\\Julia\\envr")
using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E_, x_, y_ = 0..30, 0..1, 0..1
box = IntervalBox(E_, x_, y_)
using CairoMakie
using GLMakie

In [2]:
function TM(u, p, t)
    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
    du1 = (-u[1] + p[1] * log( 1.0 + exp( (p[5] * U(u[3]) * u[2] * u[1] + p[11]  ) / (p[1]) ) ) ) / p[2]
    du2 = (1.0 - u[2])/p[3] - U(u[3])*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2])
    return SA[du1, du2, du3]
end
function jacob_TM(u, p, t)
    
    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x - p[6]) ) )
    U_y(y) = ( 50.0 * p[9] * exp(-50.0 * (y - p[7])) ) / (1.0 + exp( -50.0*(y - p[7]) ) )^2
    g(E, x, y) = exp((p[5]  * U(y) * x * E + p[11]) / p[1])
    σ_der(x) = exp( (-20.0) * (x - p[6]) )
    
    E_E = ( -1.0 + ((J * U(u[3]) * u[2] * g(u[1], u[2], u[3]))) / (1.0 + g(u[1], u[2], u[3])) ) / p[2]
    E_x = (p[5] * U(u[3]) * u[1] * g(u[1], u[2], u[3])) / (p[2] * (1.0 + g(u[1], u[2], u[3]) ))
    E_y = ( p[5]  * U_y(u[3]) * u[2] * u[1] * g(u[1], u[2], u[3]) ) / (p[2] * (1.0 + g(u[1], u[2], u[3])) )
    
    x_E = -U(u[3])*u[2]
    x_x = -1.0 / p[3] - U(u[3])*u[1]
    x_y = (-U_y(u[3])) * u[2] * u[1]
    
    y_x = 20.0 * p[10] * σ_der(u[2]) / (1.0 + σ_der(u[2]))^2
    y_y = -1.0/p[4]
    
    SMatrix{3,3}(E_E, x_E, 0.0,
        E_x, x_x, y_x,
        E_y, x_y, y_y)
end
function labels3d(idx, idy, idz)
    if idx == 1
        x = "E"
    elseif idx == 2
        x = "x"
    else
        x = "y"
    end
    if idy == 1
        y = "E"
    elseif idy == 2
        y = "x"
    else
        y = "y"
    end
    if idz == 1
        z = "E"
    elseif idz == 2
        z = "x"
    else
        z = "y"
    end
    return x,y,z
end;

In [3]:
t = 5000; tstep = 0.001; ttr = 100000
integ_set = (alg = RK4(), adaptive = false, dt = tstep)

const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305;

In [4]:
path = "C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\2М Карта спектра Ляпуновских показателей\\200x200";
filenameLS = "\\LSs_200x200.jld"
filenameu0s = "\\u0s_200x200.jld"
pathLS = path*filenameLS
pathu0s = path*filenameu0s

Λs = load(pathLS)["data"]
u0s = load(pathu0s)["data"];

In [5]:
I0range = range( -1.40, -1.709, length = 201 )
U0range = range( 0.3,  0.47, length = 201);

In [6]:
idx_I0, idx_U0 = 105, 1
I0_ = I0range[idx_I0]
U0_ = U0range[idx_U0]
println("I0:",I0_, "; U0:",  U0_)
println(Λs[idx_I0, idx_U0, :])

I0:-1.56068; U0:0.3
[0.005571183838012268, -0.162330285729737, -3.9854227511565297]


При параметрах I0:-1.61012; U0:0.3

Первый раз встречается второй аттрактор

Его начальные условия [5.029405248505961, 0.794984502437143, 0.4249747017367524]



In [7]:
I0_local = -1.6102257124
p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_local]; # -1.7096211]

In [282]:
last_point_A1 = tr_A1[end]
last_point_A2 = tr_A2[end]
println("last_point_A1: $last_point_A1")
println("last_point_A2: $last_point_A2")

last_point_A1: [3.614150444316901, 0.7995140316515059, 0.4256000782173303]
last_point_A2: [7.848022354712016, 0.7018321801423821, 0.4402545036677643]


In [8]:
ds_A1 = CoupledODEs(TM, SA[3.614150444316901, 0.7995140316515059, 0.4256000782173303], p, diffeq = integ_set)
tr_A1, _ = trajectory(ds_A1, t, Δt = tstep; Ttr = ttr)

ds_A2 = CoupledODEs(TM, SA[7.848022354712016, 0.7018321801423821, 0.4402545036677643], p, diffeq = integ_set)
tr_A2, _ = trajectory(ds_A2, t, Δt = tstep; Ttr = ttr);

In [9]:
fp, eigs, stable = fixedpoints(ds_A1, box, jacob_TM);

In [10]:
ds_A1

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  TM
 ODE solver:    RK4
 ODE kwargs:    (adaptive = false, dt = 0.001)
 parameters:    [1.58, 0.013, 0.08, 3.3, 3.07, 0.75, 0.4, 0.3, 0.305, 0.3, -1.6102257124]
 time:          105000.00004529246
 state:         [6.821994985293524, 0.6866838791713537, 0.42596145635390176]


In [11]:
ds_A2

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  TM
 ODE solver:    RK4
 ODE kwargs:    (adaptive = false, dt = 0.001)
 parameters:    [1.58, 0.013, 0.08, 3.3, 3.07, 0.75, 0.4, 0.3, 0.305, 0.3, -1.6102257124]
 time:          105000.00004529246
 state:         [8.124338914021985, 0.6552902588595779, 0.4438414263930182]


In [12]:
ts, tf = 4000000, 4500000
idx, idy, idz = 2, 1, 3
xl, yl, zl = labels3d(idx,idy, idz)

GLMakie.activate!()
f = Figure(resolution = (900, 900))
ax = Axis3(f[1, 1], xlabel = xl, ylabel = yl, zlabel = zl,
            xlabelsize = 35, ylabelsize = 35, zlabelsize = 35,
            xgridvisible = false, ygridvisible = false, zgridvisible = false)

lines!(tr_A1[ts:tf, idx], tr_A1[ts:tf, idy], tr_A1[ts:tf, idz], linewidth = 1.5, color = :green)
lines!(tr_A2[ts:tf, idx], tr_A2[ts:tf, idy], tr_A2[ts:tf, idz], linewidth = 1.5, color = :blue)

scatter!(fp[1][idx], fp[1][idy], fp[1][idz], markersize = 10, color = :red)
display(f)

GLMakie.Screen(...)